In [402]:
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polydiv
from phe import paillier
import numpy as np
import secrets

In [403]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
        self.N = sk.public_key.n
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = Polynomial.fromroots(self.S)
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = self.f#Polynomial([pk.encrypt(x) for x in self.f.coef])
        self.f_list.append(self.ef)
    
    def append_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []
        for _ in range(c):
            r.append(Polynomial([secrets.randbelow(self.N) for _ in range(self.k + 1)]))
        self.phi_i = sum([prod([self.f_list[j],r[j]]) for j in range(c)])
        #for x in self.phi_i.coef:
        #    x.obfuscate() #re-randomização dos coeficientes

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1 + self.phi_i
        #for x in self.lam_i.coef:
        #    x.obfuscate() #re-randomização dos coeficientes
    
    def decrypt(self):
        self.p = self.ep#Polynomial([self.sk.decrypt(x) for x in self.ep.coef])

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset

        #print(self.p.coef)
        #self.p = Polynomial([self.sk.public_key.encrypt(int(x)) for x in self.p.coef])
        #self.p = Polynomial([self.sk.decrypt(x) for x in self.p.coef])

        res = []
        for elem in self.S:
            degree = 0
            poly = self.p
            while poly(elem) == 0:
                degree += 1
                poly = poly.deriv(1)
            if degree > 0:
                res.append((elem, degree))
        self.intersection = res

    

Soma de polinómios encriptados: `a.ef + b.ef`

Produto de polinímio não encriptado com polinómio encriptado: `a.f * b.ef`

Derivada de polinómio encriptado: `a.ef.deriv(d)`

Avaliação de um polinómio encriptado num ponto não encriptado: `a.ef(x)`


Função da secção 5.1

In [404]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    n = len(players)
    assert n >= 2, "Not enough players"
    assert c < n, "Dishonestly colluding too large"
    assert all([player.k == players[0].k for player in players]),\
        "Differently sized private sets"
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt(pk)
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            players[i].append_f(players[(i-j) % n].ef)
        
        # Qual é o domínio?? #consoante a resposta é preciso alterar a função phi
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #c, d
        players[i].phi()

    #2
    players[0].lam_i = players[0].phi_i
    players[1].receive_l(players[0].lam_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_i)

    #5
    for player in players:
        player.decrypt()
        pass
    #6
    for player in players:
        player.multiset()

In [146]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair(None, 32)
pk.max_int = pk.nsquare

In [405]:
pk, sk = paillier.generate_paillier_keypair(None, 32)
pk.max_int = pk.nsquare

a = Player([1,2,3,4,5],sk)
b = Player([1,1,1,1,1],sk)

# Calculate private set
psi([a,b],1,pk)

In [406]:
#find private set
for x in a.intersection:
    print(x)

dp = a.p.deriv(0)
for x in a.S:
    print(dp(x))


(1, 1)
0.0
81277949206.0
15344043727168.0
448290293032530.0
5511563697117184.0


In [408]:
#testes
a = [1, 2, 1] #experimentem assim e com [1., 2., 1.,]  o polinomio força os ints a floats e lixa a encriptação!
b = Polynomial(a)
print(a)
print(b.coef)
c = [sk.public_key.encrypt(x) for x in a]
d = Polynomial([sk.public_key.encrypt(int(x)) for x in b.coef])

e = [sk.decrypt(x) for x in c]
f = Polynomial([sk.decrypt(x) for x in d.coef])
print(e)
print(f.coef)

[1, 2, 1]
[1. 2. 1.]
[1, 2, 1]
[1. 2. 1.]


In [ ]:
#compare make sure the sets are the same

In [14]:
# Teste das operações definidas em 4.2
pk, sk = paillier.generate_paillier_keypair()
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)
a.polynomial()
b.polynomial()
a.encrypt(pk)
b.encrypt(pk)

d = 2
x = 0
print(f"Soma:\n{a.ef + b.ef}")
print(f"Multiplicação:\n{a.f * b.ef}")
print(f"Derivada:\n{a.ef.deriv(d)}")
print(f"Avaliação:\n{a.ef(x)}")


Soma:
-30.0 + 37.0 x - 15.0 x**2 + 2.0 x**3
Multiplicação:
144.0 - 420.0 x + 484.0 x**2 - 285.0 x**3 + 91.0 x**4 - 15.0 x**5 +
1.0 x**6
Derivada:
-12.0 + 6.0 x
Avaliação:
-6.0


In [439]:
class MyPolynomial:
    def __init__(self, coef):
        self.coef = coef

    def empty(n):
        return MyPolynomial(np.zeros(n, dtype = int))

    def fromroots(roots):
        new_poly = MyPolynomial([])
        for x in roots:
            new_poly = new_poly.mul(MyPolynomial([-x, 1]))
        return new_poly

    def n_coef(self):
        return len(self.coef)
    
    def degree(self):
        return len(self.coef) - 1

    def reduce(self):
        last = self.n_coef()
        for i in range(len(self.coef)-1, -1, -1):
            if self.coef[i] != 0:
                break
            else:
                last -= 1
        self.coef = self.coef[:last]


    def add(self, poly):
        if self.n_coef() > poly.n_coef():
            sum = self.coef
            for i in range(poly.n_coef()):
                sum[i] += poly.coef[i]
            new_poly = MyPolynomial(sum)
        else:
            sum = poly.coef
            for i in range(self.n_coef()):
                sum[i] += self.coef[i]
            new_poly = MyPolynomial(sum)
        new_poly.reduce()
        return new_poly
    
    def mul(self, poly):
        new_coef = self.n_coef() + poly.n_coef() - 1
        new_poly = MyPolynomial.empty(new_coef)

        for i in range(self.n_coef()):
            for j in range(poly.n_coef()):
                new_poly.coef[i+j] += self.coef[i] * poly.coef[j]
        new_poly.reduce()
        return new_poly
            

In [440]:
poly1 = MyPolynomial.fromroots([1, -1])
print(poly1.coef)

[]
